In [ ]:
!pip install llama-index-llms-openai-like
%pip install llama-index
%pip install llama-index-retrievers-bm25
%pip install llama-index-tools-duckduckgo

In [27]:
import requests
import json
import re


os.environ["OPENAI_API_KEY"] = "sk-xxxxxxxx"

## Can reasoning model be reasonable medical diagnoser

### Step 1 - Use a distilled version of R1

We use Llama8B R1 as a reasoning engine.


#### How to start Llama8B-R1 locally

Use [vllm](https://docs.vllm.ai/en/stable/)

Download a quantized version of the [model](https://huggingface.co/lmstudio-community/DeepSeek-R1-Distill-Llama-8B-GGUF/tree/main)

Run it with the following command 

```shell
vllm serve './DeepSeek-R1-Distill-Llama-8B-Q4_K_M.gguf' --tokenizer deepseek-ai/DeepSeek-R1-Distill-Llama-8B --max-model-len 8000 --enable-auto-tool-choice --tool-call-parser llama3_json
```

You should be able to access it with an API call 

```shell
curl http://localhost:8000/v1/completions     -H "Content-Type: application/json"     -d '{
        "model": "/home/.../.../DeepSeek-R1-Distill-Llama-8B-Q4_K_M.gguf",
        "prompt": "Create a joke with 2 animals of your choice. Think step by step about the animal attributes and what could link them for a joke. ",
        "max_tokens": 7800,
        "temperature": 0.8
    }'

```

In [3]:
model_name = "/home/amor/Downloads/DeepSeek-R1-Distill-Llama-8B-Q4_K_M.gguf"

In [19]:
PROMPT = "Create a joke with 2 animals of your choice. First list famous jokes and try to adapt them."

def call_r1_small(prompt, max_tok=7800):
    config = {
            "model": model_name,
            "prompt": prompt, 
            "max_tokens": max_tok,
            "temperature": 0.8
        }
    
    rez = requests.post("http://localhost:8000/v1/completions",
                        data=json.dumps(config), headers={"Content-Type": "application/json"})
    full_answer = rez.json()["choices"][0]["text"]
    pattern = re.compile("</think>")
    final_answer = full_answer[pattern.search(full_answer).span()[1]:]
    return full_answer, final_answer


In [8]:
a, b = call_r1_small(PROMPT, max_tok=7500)

In [9]:
print(b)



**Joke:**

*Why did the duck argue with the scarecrow?*

*Because it thought the scarecrow was a duck!* 🐝️🧅

This joke plays on the scarecrow's appearance, which is often resembles a human, and the duck's confusion between the two, creating a humorous twist.


## Step 2 - Use it for a clinical reasoning

Create a prompt with a medical case

In [10]:
NURSE_EXAMPLE_PROMPT = """
   A user comes with a pediatric issue. Think about the possible reason of the issue and try to 
   provide a list of bullet points to improve the user issues.

   <user_issue> 
   My baby has vomited 3 times this night, should I be worried ? He had only milk as his dinner.
   </user_issue>

   Solve this issue by thinking step by step.
   
   """
full_answer, final_answer = call_r1_small(NURSE_EXAMPLE_PROMPT, max_tok=7500)

In [11]:
print(final_answer)



**Answer:**

If your baby has vomited three times in one night and has only had milk as dinner, it's natural to be concerned. Here's a structured approach to address this situation:

1. **Assess for Dehydration:**
   - Check for signs such as dry mouth, reduced urination, or lethargy. If these are present, contact a healthcare provider immediately.

2. **Monitor for Other Symptoms:**
   - Look for symptoms like fever, diarrhea, or rash. These could indicate more serious issues like gastroenteritis or an infection.

3. **Evaluate Feeding and Allergies:**
   - Ensure the baby's feeding practices are appropriate. If a new food or formula was introduced, consider an allergic reaction.

4. **Consider Reflux or Developmental Issues:**
   - Discuss with a pediatrician if reflux or developmental milestones might be contributing factors.

5. **Check for Environmental or Stress Factors:**
   - Investigate if environmental factors like overheating or stress could be causing discomfort.

6. **Pr

In [14]:
from llama_index.llms.openai_like import OpenAILike

llm = OpenAILike(model=model_name, api_base="http://localhost:8000/v1", api_key="fake", 
                 max_tokens=7500, temperature=0.7)


In [ ]:

response = llm.complete("Hello World!")
print(str(response))

In [75]:
from llama_index.retrievers.bm25 import BM25Retriever
from llama_index.core import SimpleDirectoryReader
from llama_index.core.node_parser import SentenceSplitter
import Stemmer


documents = SimpleDirectoryReader("./data/").load_data()
splitter = SentenceSplitter(chunk_size=512)
nodes = splitter.get_nodes_from_documents(documents)

def load_retriever_bm25(nodes):
    bm25_retriever = BM25Retriever.from_defaults(
        nodes=nodes,
        similarity_top_k=3,
        # Optional: We can pass in the stemmer and set the language for stopwords
        # This is important for removing stopwords and stemming the query + text
        # The default is english for both
        stemmer=Stemmer.Stemmer("english"),
        language="english",
    )
    return bm25_retriever


retriever = load_retriever_bm25(nodes)

In [26]:
from llama_index.tools.duckduckgo import DuckDuckGoSearchToolSpec
from llama_index.agent.openai import OpenAIAgent
import os
from llama_index.core.agent import ReActAgent
from llama_index.llms.openai import OpenAI
from llama_index.core.llms import ChatMessage
from llama_index.core.tools import BaseTool, FunctionTool
from typing import Optional


parent_llm = OpenAI()



def search_medical_knowledge(query_word: str) -> str:
    """Search about a mdeical fact in the verified medical database knowledge using a single word"""
    results = retriever.retrieve(query_word)
    return "\n".join(f"<result>{r.text}</result>" for r in results)

def ask_pediatrician(issue: str, patient_context: str) -> str:
    """Ask a pediatrician about a case. 
    issue describes the symptoms, context allow to describe the medical hisotry and details of the patient 
    """
    PROMPT_PEDIATRICIAN = f"""
        You are a pediatrician bot
        Solve this user issue. 

        <patient_context> {patient_context} </patient_context>
        
        <issue>{issue}</issue>
        
        Your answer should list a set of hypothesis about the rootcause of the issue and rank it with medical risk levels
        
        Answer in a list of structured format
        
        <hypothesis><root_cause> ... <root_cause><risk> 1/5 </risk> </hypothesis>
        
        """
    text = llm.complete(PROMPT_PEDIATRICIAN).text
    pattern = re.compile("</think>")
    final_answer = text[pattern.search(text).span()[1]:]
    return final_answer


def ask_clarification_to_parent(question: str) -> str:
    """Ask clarification to the parent about a missing information on the baby"""
    PRMOPT = f"""You are a prent of a young child of 11 month who is voimitting. 
    If you receive a question about relevant information, you must provide relevant answers from the context section only. 
    Here is the context of the child 
    <context>
    - Last breast milk was 2 hours ago, he vomited 3 times since
    - No issues of vomiting in the past
    - The child did not eat anything other than the breast milk
    - The child has no fever
    - The child is not sick
    - The child could sleep
    </context>
    Now answer the question : 
    Doctor: {question}
    Parent:  
    """
    return parent_llm.complete(PRMOPT).text



search_medical_knowledge_tool = FunctionTool.from_defaults(fn=search_medical_knowledge)
ask_pediatrician_tool = FunctionTool.from_defaults(fn=ask_pediatrician)
ask_parent_tool = FunctionTool.from_defaults(fn=ask_clarification_to_parent)
tool_spec = DuckDuckGoSearchToolSpec()
agent = OpenAIAgent.from_tools(tool_spec.to_tool_list() + [search_medical_knowledge_tool, ask_pediatrician_tool, ask_parent_tool], 
                               verbose=True)

PROMPT_BABY = """
You are a bot receiving demands for help from Parents. 
You can use multiple tools to inform as best as you can the parent about the issue from their child. 

Solve this user issue. 

<context> The child is 11 months old with no know history of health issue.<context> 

<issue>My baby has vomited 3 times this night, should I be worried ? He had only breast milk as his dinner.</issue>

 
Don't use medical terminologies or word that are hard to understand.
List all the possible causes of danger with clear and precise indication about when to reach for a doctor. Don't use vague terms like unwell, unusual or bad.
Don't use vague elements like "trust your instincts as a parent" or any non scientific elements

For all your hypothesis from the pediatrician, ask clarifying elements to the user before delivering the final list of information.
WHen receiving feedback from the user, re-assess your list to see if some options are discarded
If you keep high risk hypothesis in your list, they should be backed by feedback of the users.
While your list contains high risk answers, you must gather more details.
"""

resp = agent.chat(PROMPT_BABY)


Added user message to memory: 
You are a bot receiving demands for help from Parents. 
You can use multiple tools to inform as best as you can the parent about the issue from their child. 

Solve this user issue. 

<context> The child is 11 months old with no know history of health issue.<context> 

<issue>My baby has vomited 3 times this night, should I be worried ? He had only breast milk as his dinner.</issue>

 
Don't use medical terminologies or word that are hard to understand.
List all the possible causes of danger with clear and precise indication about when to reach for a doctor. Don't use vague terms like unwell, unusual or bad.
Don't use vague elements like "trust your instincts as a parent" or any non scientific elements

For all your hypothesis from the pediatrician, ask clarifying elements to the user before delivering the final list of information.
WHen receiving feedback from the user, re-assess your list to see if some options are discarded
If you keep high risk hypoth

## Conclusion 

It is hard to steer a React Agent to do exactly what you want.

For this use case, a workflow could work just as well with

- Reasoning model diagnosis
- Hypothesis generation :
- Medical knowledge search : hypothesis to symptoms and antecedants 
- Additional medical history inquiry for hypothesis infirmation (asked to patient)
- Final hypothesis list
- Return adapted answer to the patient


